In [1]:
import pandas as pd

In [2]:
with open('meeting.txt', 'r', encoding='utf-8') as file:
    data = file.read()

print(f"Number of tokens (words): {len(data.split())}")
print(data[:500]) 


Number of tokens (words): 6614
can record. And we don't have a ton of items to get to. And I might be able to do one that might be fun if we have a little bit of time. So corporate events, they, I think I saw a little, I put this in Slack and I saw a little bit of kind of noise around it, which was good. The nutshell here is as we've kind of restructured and tried different things. The event support that we need isn't as nailed down as it needs to be. So the current tactic that we're going with is go-to-market team, signs up 


In [3]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')
nltk.download('punk')
sentences = sent_tokenize(data)

print(sentences)

['can record.', "And we don't have a ton of items to get to.", 'And I might be able to do one that might be fun if we have a little bit of time.', 'So corporate events, they, I think I saw a little, I put this in Slack and I saw a little bit of kind of noise around it, which was good.', "The nutshell here is as we've kind of restructured and tried different things.", "The event support that we need isn't as nailed down as it needs to be.", "So the current tactic that we're going with is go-to-market team, signs up and kind of sponsors that event.", 'So you support as a PMM, your campaign manager does the campaigns for that event, etc, etc.', "I don't see anyone in the, maybe there are comments in the issue.", "I don't see the header updated yet.", 'I thought we had in Slack sort of farm to each one of them out.', 'So I guess the next, so it looks like tie put in some folks.', "It looks like this looks good so that let's see needs support from GTM teams.", 'So I guess the ask would be t

[nltk_data] Downloading package punkt_tab to /home/sachin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Error loading punk: Package 'punk' not found in index


In [4]:
text = ' '.join(sentences)
print(text)
print(len(text)) 

can record. And we don't have a ton of items to get to. And I might be able to do one that might be fun if we have a little bit of time. So corporate events, they, I think I saw a little, I put this in Slack and I saw a little bit of kind of noise around it, which was good. The nutshell here is as we've kind of restructured and tried different things. The event support that we need isn't as nailed down as it needs to be. So the current tactic that we're going with is go-to-market team, signs up and kind of sponsors that event. So you support as a PMM, your campaign manager does the campaigns for that event, etc, etc. I don't see anyone in the, maybe there are comments in the issue. I don't see the header updated yet. I thought we had in Slack sort of farm to each one of them out. So I guess the next, so it looks like tie put in some folks. It looks like this looks good so that let's see needs support from GTM teams. So I guess the ask would be to work with your GTM teams. So let me ask

In [5]:
def split_text_into_chunks(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap  # Move forward but keep overlap
    return chunks


In [6]:
chunks = split_text_into_chunks(text, chunk_size=1000, overlap=200)

# Optional: print a few chunks
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk)
    print()


--- Chunk 1 ---
can record. And we don't have a ton of items to get to. And I might be able to do one that might be fun if we have a little bit of time. So corporate events, they, I think I saw a little, I put this in Slack and I saw a little bit of kind of noise around it, which was good. The nutshell here is as we've kind of restructured and tried different things. The event support that we need isn't as nailed down as it needs to be. So the current tactic that we're going with is go-to-market team, signs up and kind of sponsors that event. So you support as a PMM, your campaign manager does the campaigns for that event, etc, etc. I don't see anyone in the, maybe there are comments in the issue. I don't see the header updated yet. I thought we had in Slack sort of farm to each one of them out. So I guess the next, so it looks like tie put in some folks. It looks like this looks good so that let's see needs support from GTM teams. So I guess the ask would be to work with your GTM team

In [7]:
import requests

def summarize_chunk(chunk, model="llama3.2:3b"):
    prompt = f"Summarize the following meeting transcript:\n\n{chunk}"
    
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False
            }
        )
        response.raise_for_status()
        return response.json().get("response", "⚠️ No response received.")
    
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [8]:
summaries = []

for idx, chunk in enumerate(chunks):
    print(f"--- Summarizing Chunk {idx + 1} ---")
    summary = summarize_chunk(chunk)
    print(summary)
    print()
    summaries.append(summary)


--- Summarizing Chunk 1 ---
The meeting transcript appears to be a summary of a discussion regarding corporate events and event support. The key points are:

* There is still uncertainty about the specific event support needed.
* The current approach is for the go-to-market team to sponsor and sign up for events, with other teams providing support (e.g., campaign management).
* It seems that there was an attempt to assign a specific task to each person involved, but this hasn't been fully completed.
* The next step is for the event support to be provided by the GTM teams, which would require collaboration and support from those teams.

Overall, it seems like the team is still in the process of figuring out the best approach for corporate events and gathering support for the necessary tasks.

--- Summarizing Chunk 2 ---
This appears to be a meeting transcript about GTM (Growth Team Management) teams and their support for a project. Here's a summary:

- The speaker is discussing next ste

In [9]:
import json

# Create a list of chunk summary entries
data = []

for i, summary in enumerate(summaries, start=1):
    data.append({
        "chunk": f"Chunk {i}",
        "summary": summary
    })

# Save to JSON file
with open("chunk_summaries.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Summaries saved to 'chunk_summaries.json'")


✅ Summaries saved to 'chunk_summaries.json'


Action Items

In [10]:
def extract_action_items(summaries):
    context = "\n\n".join([f"{s['chunk']}: {s['summary']}" for s in summaries])
    
    prompt = (
        "From the following meeting summaries, extract a list of clear, actionable tasks "
        "or to-do items discussed. Include who is responsible if mentioned.\n\n"
        f"{context}\n\n"
        "List the tasks in bullet points or numbered list."
    )

    return summarize_chunk(prompt)

In [11]:
import json

# Load summaries
with open("chunk_summaries.json", "r", encoding="utf-8") as f:
    summaries = json.load(f)

# Extract action items
action_items = extract_action_items(summaries)

print("📌 Action Items / TODOs:")
print(action_items)


📌 Action Items / TODOs:
Here are the tasks mentioned in the transcript:

1. **Create an industry resource page** comparing DevOps tools
2. **Design a new website** with a fresh design concept
3. **Develop a software factory concept**
4. **Brainstorm slogans or taglines for GitLab** (including "secure the software factory" phrase)
5. **Present ideas and work on messaging framework** for a blog post
6. **Create marketing materials** (e.g., slogans, taglines) for GitLab
7. **Develop a new product or idea**
8. **Participate in a NASCAR partnership discussion** with GitLab
9. **Collaborate on bike racing analogies and speed optimization strategies**
10. **Plan fun activities and outings** (including movie nights)
11. **Prioritize tasks and focus on speed over risk**

Note that some of these tasks may be ongoing or iterative, rather than single-point tasks. Additionally, the transcript does not provide a comprehensive list of all tasks, but rather highlights specific discussions and ideas re

In [12]:
with open("action_items.txt", "w", encoding="utf-8") as f:
    f.write(action_items)
